# 对于50个APP，生成用户间共现网络

In [1]:
import numpy as np
import pandas as pd

In [2]:
readAppCoScale = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_coScale.csv')[['appID', 'coScale_logReci']].set_index('appID')
dictAppScore = readAppCoScale.to_dict() #索引，为了加快速度

In [3]:
# 读取user_apps
readUserApps = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_apps.csv', names=['userID', 'appList'])
dict_userApps = {}
def getDictUserApps(x):
     dict_userApps[x[0]] = x[1][1:-1].split(', ')
readUserApps.apply(getDictUserApps, axis=1)
readUserApps.head()

,userID,appList
0,1,"[357, 360, 362, 365, 375, 441, 480, 481, 482, ..."
1,9,"[538, 602, 604]"
2,10,"[357, 359, 365, 383, 480, 481, 482, 483, 484, ..."
3,12,"[480, 481, 482, 483, 484, 485, 486, 487, 488, ..."
4,14,"[362, 480, 481, 482, 483, 484, 485, 486, 487, ..."


In [16]:
# 读取训练文件
dataNetAll = pd.merge(pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv"), pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv"), on='creativeID')

In [4]:
def getScore(list):
    score = 0
    for l in list:
        score += dictAppScore['coScale_logReci'][int(l)]
    return score
def cal_coScale(user0, user1):
    list0 = dict_userApps[user0]
    list1 = dict_userApps[user1]
    interset = np.intersect1d(list0, list1)
    length = len(interset)
    if(length == 0): 
        return 0
    else: 
#         return getScore(interset)
#         return getScore(interset)/length
        return getScore(np.intersect1d(list0, list1))/getScore(np.union1d(list0, list1))

In [31]:
def writeNet(appID):
    # 取某个appID的用户，分为1和0两个部分
    dataNet = dataNetAll[dataNetAll['appID']==appID][['label','userID']].copy()
    dataNet_1 = dataNet[dataNet['label']==1]
    dataNet_0 = dataNet[dataNet['label']==0]
    ## 前期准备
    listUsers1 = list(dataNet_1['userID'])
    if len(listUsers1) > 1500:
        listUsers1 = listUsers1[::int(len(listUsers1)/1500)] #控制listUser1的长度，如果大于1500，就间隔抽样1500个
    sampler = np.random.permutation(len(listUsers1))
    index = dataNet_0['userID'].index[sampler] #重排序后的索引
    listUsers0 = list(dataNet_0['userID'].ix[index])
    # node
    ## node1
    f_node = open('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\' + str(appID) + '_node.csv', 'w')
    f_node.write('Id,Label,partition\r\n')
    for i in range(len(listUsers1)):
        f_node.write(str(listUsers1[i]) + ',' + str(listUsers1[i]) + ',' + '1' + '\r\n')
    ## node0
    for i in range(len(listUsers0)):
        f_node.write(str(listUsers0[i]) + ',' + str(listUsers0[i]) + ',' + '0' + '\r\n')
    f_node.close()
    # line
    f_line = open('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\' + str(appID) + '_line.csv', 'w')
    f_line.write('Source,Target,weight,partition\r\n')
    threshold_score = 0.3
    # 1网络
    for i in range(len(listUsers1)):
        if dict_userApps.has_key(listUsers1[i]) == False: continue
        for j in range(i, len(listUsers1)):
            if dict_userApps.has_key(listUsers1[j]) == False: continue
            if listUsers1[i] == listUsers1[j]: continue
            score = cal_coScale(listUsers1[i], listUsers1[j])
            if score > threshold_score:
                f_line.write(str(listUsers1[i]) + "," + str(listUsers1[j]) + ","
                             + str(score) + ',' + "1\r\n")
    # 0网络
    for i in range(len(listUsers0)):
        if dict_userApps.has_key(listUsers0[i]) == False: continue
        for j in range(i, len(listUsers0)):
            if dict_userApps.has_key(listUsers0[j]) == False: continue
            if listUsers0[i] == listUsers0[j]: continue
            score = cal_coScale(listUsers0[i], listUsers0[j])
            if score > threshold_score:
                f_line.write(str(listUsers0[i]) + "," + str(listUsers0[j]) + ","
                             + str(score) + ',' + "0\r\n")
    # 1-0网络
    for i in range(len(listUsers1)):
        if dict_userApps.has_key(listUsers1[i]) == False: continue
        for j in range(i, len(listUsers0)):
            if dict_userApps.has_key(listUsers0[j]) == False: continue
            score = cal_coScale(listUsers1[i], listUsers0[j])
            if score > threshold_score:
                f_line.write(str(listUsers1[i]) + "," + str(listUsers0[j]) + ","
                             + str(score) + ',' + "10\r\n")
    f_line.close()

In [42]:
writeNet(75)

In [43]:
# 获得targetID名单
dataAppID = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv')
appIDs = set(dataAppID['appID'])
for id in appIDs:
    print id
#     writeNet(id)

389
262
391
137
14
271
472
146
150
25
283
284
286
160
278
419
420
421
428
304
434
442
319
195
68
198
327
328
75
205
206
336
465
83
84
88
100
218
350
293
123
356
229
360
105
109
113
116
127
383
